In [1]:
import cupy as cp
import cusignal 
from scipy import signal
import numpy as np

### Resample

In [2]:
start = 0
stop = 10
num = int(1e8)
resample_num = int(1e5)

cx = np.linspace(start, stop, num, endpoint=False) 
cy = np.cos(-cx**2/6.0)

In [3]:
%%timeit
cf = signal.resample(cy, resample_num, window=('kaiser', 0.5))

14.3 s ± 438 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [4]:
gx = cp.linspace(start, stop, num, endpoint=False)
gy = cp.cos(-gx**2/6.0)

In [5]:
%%timeit
gf = cusignal.resample(gy, resample_num, window=('kaiser',0.5))

71.1 ms ± 2.45 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Resample Poly

In [6]:
start = 0
stop = 10
num = int(1e8)
resample_up = 2
resample_down = 3

cx = np.linspace(start, stop, num, endpoint=False) 
cy = np.cos(-cx**2/6.0)

In [7]:
%%timeit
cf = signal.resample_poly(cy, resample_up, resample_down, window=('kaiser', 0.5))

2.41 s ± 10 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
gx = cp.linspace(start, stop, num, endpoint=False)
gy = cp.cos(-gx**2/6.0)

In [9]:
%%timeit
cf = cusignal.resample_poly(gy, resample_up, resample_down, window=('kaiser', 0.5), use_numba=True)

32.3 ms ± 110 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
cf = cusignal.resample_poly(gy, resample_up, resample_down, window=('kaiser', 0.5), use_numba=False)

17.3 ms ± 14.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### FIR Filter Design with Window

In [11]:
numtaps = int(1e8)
f1, f2 = 0.1, 0.2

In [12]:
%%timeit
cfirwin = signal.firwin(numtaps, [f1, f2], pass_zero=False)

13.5 s ± 20.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit
gfirwin = cusignal.firwin(numtaps, [f1, f2], pass_zero=False)

149 ms ± 21.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Correlate

In [14]:
sig = np.random.rand(int(1e8))
sig_noise = sig + np.random.randn(len(sig))

In [15]:
%%timeit
ccorr = signal.correlate(sig_noise, np.ones(128), mode='same') / 1e6

5.06 s ± 5.65 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
sig = cp.random.rand(int(1e8))
sig_noise = sig + cp.random.randn(len(sig))

In [17]:
%%timeit
gcorr = cusignal.correlate(sig_noise, cp.ones(128), mode='same') / 1e6

73.1 ms ± 418 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Convolve

In [18]:
sig = np.random.rand(int(1e8))
win = signal.windows.hann(int(1e3))

In [19]:
%%timeit
cconv = signal.convolve(sig, win, mode='same') / np.sum(sig)

15.7 s ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
sig = cp.random.rand(int(1e8))
win = cusignal.hann(int(1e3))

In [21]:
%%timeit
gconv = cusignal.convolve(sig, win, mode='same') / cp.sum(win)

72.9 ms ± 20.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Convolution using the FFT Method

In [22]:
csig = np.random.randn(int(1e8))

In [23]:
%%timeit
cautocorr = signal.fftconvolve(csig, csig[::-1], mode='full')

27.3 s ± 171 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
gsig = cp.random.randn(int(1e8))

In [25]:
%%timeit
gautocorr = cusignal.fftconvolve(gsig, gsig[::-1], mode='full')

128 ms ± 192 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Wiener Filter on N-Dimensional Array

In [26]:
csig = np.random.rand(int(1e8))

In [27]:
%%timeit
cfilt = signal.wiener(csig)

3.57 s ± 3.69 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
gsig = cp.random.randn(int(1e8))

In [29]:
%%timeit
gfilt = cusignal.wiener(gsig)

256 ms ± 43.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Perform 1-D Hilbert Transform

In [30]:
csig = np.random.rand((int(1e8)))

In [31]:
%%timeit
chtrans = signal.hilbert(csig)

11.2 s ± 79 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
gsig = cp.random.rand(int(1e8))

In [33]:
%%timeit
ghtrans = cusignal.hilbert(gsig)

71.3 ms ± 1.49 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Perform 2-D Hilbert Transform

In [34]:
csig = np.random.rand(int(1e4), int(1e4))

In [35]:
%%timeit
chtrans2d = signal.hilbert2(csig)

6.26 s ± 6.31 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
gsig = cp.random.rand(int(1e4), int(1e4))

In [37]:
%%timeit
ghtrans2d = cusignal.hilbert2(gsig)

84.5 ms ± 2.02 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Perform 2-D Convolution and Correlation

In [38]:
csig = np.random.rand(int(1e4), int(1e4))
filt = np.random.rand(5,5)

In [39]:
%%timeit
grad = signal.convolve2d(csig, filt)

8.26 s ± 41.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
gsig = cp.random.rand(int(1e4), int(1e4))
gfilt = cp.random.rand(5,5)

In [41]:
%%timeit
ggrad = cusignal.convolve2d(gsig, gfilt, use_numba=True)

45.8 ms ± 811 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%%timeit
ggrad = cusignal.convolve2d(gsig, gfilt, use_numba=False)

27.1 ms ± 376 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [43]:
%%timeit
grad = signal.correlate2d(csig, filt)

8.29 s ± 21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%%timeit
ggrad = cusignal.correlate2d(gsig, gfilt, use_numba=True)

46 ms ± 1e+03 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [45]:
%%timeit
ggrad = cusignal.correlate2d(gsig, gfilt, use_numba=False)

27.1 ms ± 471 ns per loop (mean ± std. dev. of 7 runs, 100 loops each)


### Decimate

In [46]:
csig = np.random.rand(int(1e8))

In [47]:
%%timeit
cdecimate = signal.decimate(csig, 3, ftype='fir')

2.13 s ± 2.73 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [48]:
gsig = cp.random.rand(int(1e8))

In [49]:
%%timeit
gdecimate = cusignal.decimate(gsig, 3)

15.9 ms ± 44.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
